In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf # we used tensorflow 2.6.0
from sklearn.model_selection import train_test_split
from tensorflow import keras

# seeds to make results reproducible
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
# auxiliary functions for data augmentation

def rotate90(df):
    df_90 = df.copy()
    df_90['mfc1'] = df['mfc4']
    df_90['mfc2'] = df['mfc1']
    df_90['mfc3'] = df['mfc2']
    df_90['mfc4'] = df['mfc3']
    for i in [1, 2, 3]:
        df_90[f'x{i}'] = 1 - df[f'y{i}']
        df_90[f'y{i}'] = df[f'x{i}']
    df_90['angle'] = df['angle'] + 90
    return df_90

def flip(df):
    df_flip = df.copy()
    df_flip['mfc1'] = df['mfc2']
    df_flip['mfc2'] = df['mfc1']
    df_flip['mfc3'] = df['mfc4']
    df_flip['mfc4'] = df['mfc3']
    for i in [1, 2, 3]:
        df_flip[f'x{i}'] = 1 - df[f'x{i}']
        df_flip[f'y{i}'] = df[f'y{i}']
    df_flip['flipped'] = True
    return df_flip

In [ ]:
# load data
df = pd.read_csv('data.csv', index_col=0)

# augment data
df['angle'] = 0
df = df.append(rotate90(df))
df = df.append(rotate90(df[(df['angle'] == 90)]))
df = df.append(rotate90(df[(df['angle']==180)]))
df['flipped'] = False
df = df.append(flip(df))

# keep only 1-leakage samples (only these are considered in this paper)
df = df[df['n'] == 1]

# normalize total flow so that x_1 + x_2 + x_3 + x_4 = 1
df['mfcsum'] = df['mfc1'] + df['mfc2'] + df['mfc3'] + df['mfc4']
df[['mfc1', 'mfc2', 'mfc3', 'mfc4']] = df[['mfc1', 'mfc2', 'mfc3', 'mfc4']].div(df['mfcsum'], axis=0)

# remove obsolete columns
df.drop(['s1', 'x2', 'y2', 's2', 'x3', 'y3', 's3', 'n', 'mfcsum'], axis=1, inplace=True)

# apply coordinate transformation to get (y_1, y_2) in [-1, 1] x [-1, 1]
df['x1'] = df['x1'].map(lambda z: 2 * z - 1)
df['y1'] = df['y1'].map(lambda z: -2 * z + 1)

In [ ]:
# get training and test data
X_train = df.loc[(df['split'] == 'train') & (df['angle'] == 0) & (df['flipped'] == False), ['mfc1', 'mfc2', 'mfc3', 'mfc4']].values.astype(np.float32)
X_train_augmented = df.loc[(df['split'] == 'train'), ['mfc1', 'mfc2', 'mfc3', 'mfc4']].values.astype(np.float32)
X_test = df.loc[(df['split'] == 'test') & (df['angle'] == 0) & (df['flipped'] == False), ['mfc1', 'mfc2', 'mfc3', 'mfc4']].values.astype(np.float32)
X_test_augmented = df.loc[(df['split'] == 'test'), ['mfc1', 'mfc2', 'mfc3', 'mfc4']].values.astype(np.float32)
Y_train = df.loc[(df['split'] == 'train') & (df['angle'] == 0) & (df['flipped'] == False), ['x1', 'y1']].values.astype(np.float32)
Y_train_augmented = df.loc[(df['split'] == 'train'), ['x1', 'y1']].values.astype(np.float32)
Y_test = df.loc[(df['split'] == 'test') & (df['angle'] == 0) & (df['flipped'] == False), ['x1', 'y1']].values.astype(np.float32)
Y_test_augmented = df.loc[(df['split'] == 'test'), ['x1', 'y1']].values.astype(np.float32)

# split validation data from training data
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)
X_train_augmented, X_val_augmented, Y_train_augmented, Y_val_augmented = train_test_split(X_train_augmented, Y_train_augmented, test_size=0.1, random_state=0)

In [ ]:
# custom hidden layer for equivariant model
class EquivariantHidden(keras.layers.Layer):
    def __init__(self):
        super(EquivariantHidden, self).__init__()
        
    def build(self, input_shape):
        initializer = tf.keras.initializers.RandomNormal(stddev=0.2)
        self.a = self.add_weight(shape=(), initializer=initializer, trainable=True)
        self.b = self.add_weight(shape=(), initializer=initializer, trainable=True)
        self.c = self.add_weight(shape=(), initializer=initializer, trainable=True)
        
    def call(self, inputs):
        return tf.nn.elu(self.a * inputs + \
                         self.b * (tf.gather(inputs, [3, 2, 1, 0], axis=1) + tf.gather(inputs, [1, 0, 3, 2], axis=1)) + \
                         self.c * tf.gather(inputs, [2, 3, 0, 1], axis=1))
    
# custom output layer for equivariant model
class EquivariantOutput(keras.layers.Layer):
    def __init__(self):
        super(EquivariantOutput, self).__init__()
        
    def build(self, input_shape):
        initializer = tf.keras.initializers.RandomNormal(stddev=0.2)
        self.d = self.add_weight(shape=(), initializer=initializer, trainable=True)
        
    def call(self, inputs):
        return self.d * tf.concat([tf.reduce_sum(inputs * [1, -1, -1, 1], axis=1, keepdims=True),
                                   tf.reduce_sum(inputs * [-1, -1, 1, 1], axis=1, keepdims=True)], axis=1)

In [ ]:
# get and print best hyperparameter configurations from numerical results
df_hp = pd.read_csv('results.csv')
tmp = df_hp.loc[(df_hp['type']=='eqv') & (df_hp['augmented']==False), 'mse_val']
index_eqv = tmp.index[tmp.argmin()]
print('\nEquivariant without data augmentation:\n')
print(df_hp.loc[index_eqv])
tmp = df_hp.loc[(df_hp['type']=='eqv') & (df_hp['augmented']==True), 'mse_val']
index_eqv_aug = tmp.index[tmp.argmin()]
print('\nEquivariant with data augmentation:\n')
print(df_hp.loc[index_eqv_aug])
tmp = df_hp.loc[(df_hp['type']=='fcnn') & (df_hp['augmented']==False), 'mse_val']
index_fcnn = tmp.index[tmp.argmin()]
print('\nStandard FCNN without data augmentation:\n')
print(df_hp.loc[index_fcnn])
tmp = df_hp.loc[(df_hp['type']=='fcnn') & (df_hp['augmented']==True), 'mse_val']
index_fcnn_aug = tmp.index[tmp.argmin()]
print('\nStandard FCNN with data augmentation:\n')
print(df_hp.loc[index_fcnn_aug])

In [ ]:
# train eqv model without data augmentation
hp = df_hp.loc[index_eqv]
models = []
models_mse = []
for i in range(10):
    models.append(keras.models.Sequential([EquivariantHidden() for _ in range(hp['depth'])]))
    models[-1].add(EquivariantOutput())
    models[-1].compile(loss='mse', optimizer=keras.optimizers.Nadam(learning_rate=hp['learning_rate']))
    early_stopping = keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)
    models[-1].fit(X_train, Y_train, batch_size=hp['batch_size'], epochs=hp['epochs'], validation_data=(X_val, Y_val), callbacks=early_stopping, verbose=0)
    models_mse.append(models[-1].evaluate(X_val, Y_val))
model_eqv = models[np.argmin(models_mse)]

In [ ]:
# train eqv model with data augmentation
hp = df_hp.loc[index_eqv_aug]
models = []
models_mse = []
for i in range(10):
    models.append(keras.models.Sequential([EquivariantHidden() for _ in range(hp['depth'])]))
    models[-1].add(EquivariantOutput())
    models[-1].compile(loss='mse', optimizer=keras.optimizers.Nadam(learning_rate=hp['learning_rate']))
    early_stopping = keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)
    models[-1].fit(X_train_augmented, Y_train_augmented, batch_size=hp['batch_size'], epochs=hp['epochs'], validation_data=(X_val, Y_val), callbacks=early_stopping, verbose=0)
    models_mse.append(models[-1].evaluate(X_val, Y_val))
model_eqv_aug = models[np.argmin(models_mse)]

In [ ]:
# train fcnn model with data augmentation
hp = df_hp.loc[index_fcnn]
models = []
models_mse = []
for i in range(10):
    models.append(keras.models.Sequential([keras.layers.Dense(hp['width'], activation='elu') for _ in range(hp['depth'])]))
    models[-1].add(keras.layers.Dense(2))
    models[-1].compile(loss='mse', optimizer=keras.optimizers.Nadam(learning_rate=hp['learning_rate']))
    early_stopping = keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)
    models[-1].fit(X_train, Y_train, batch_size=hp['batch_size'], epochs=hp['epochs'], validation_data=(X_val, Y_val), callbacks=early_stopping, verbose=0)
    models_mse.append(models[-1].evaluate(X_val, Y_val))
model_fcnn = models[np.argmin(models_mse)]

In [ ]:
# train fcnn model with data augmentation
hp = df_hp.loc[index_fcnn_aug]
models = []
models_mse = []
for i in range(10):
    models.append(keras.models.Sequential([keras.layers.Dense(hp['width'], activation='elu') for _ in range(hp['depth'])]))
    models[-1].add(keras.layers.Dense(2))
    models[-1].compile(loss='mse', optimizer=keras.optimizers.Nadam(learning_rate=hp['learning_rate']))
    early_stopping = keras.callbacks.EarlyStopping(patience=100, restore_best_weights=True)
    models[-1].fit(X_train_augmented, Y_train_augmented, batch_size=hp['batch_size'], epochs=hp['epochs'], validation_data=(X_val, Y_val), callbacks=early_stopping, verbose=0)
    models_mse.append(models[-1].evaluate(X_val, Y_val))
model_fcnn_aug = models[np.argmin(models_mse)]

In [ ]:
# prepare model inputs and compute predictions for figure 5
const_rates = 0.25 * np.ones(51)
increasing_rates = np.arange(0.0, 0.51, 0.01)
decreasing_rates = np.arange(0.50, -0.01, -0.01)
data_13 = np.c_[increasing_rates, const_rates, decreasing_rates, const_rates]
data_24 = np.c_[const_rates, increasing_rates, const_rates, decreasing_rates]
predictions_eqv_13 = model_eqv.predict(data_13)
predictions_eqv_24 = model_eqv.predict(data_24)
predictions_eqv_aug_13 = model_eqv_aug.predict(data_13)
predictions_eqv_aug_24 = model_eqv_aug.predict(data_24)
predictions_fcnn_13 = model_fcnn.predict(data_13)
predictions_fcnn_24 = model_fcnn.predict(data_24)
predictions_fcnn_aug_13 = model_fcnn_aug.predict(data_13)
predictions_fcnn_aug_24 = model_fcnn_aug.predict(data_24)

In [ ]:
# reproduce figure 5a
plt.figure(figsize=(2, 2))
plt.scatter(predictions_eqv_13[:, 0], predictions_eqv_13[:, 1], label='1 $\leftrightarrow$ 3')
plt.scatter(predictions_eqv_24[:, 0], predictions_eqv_24[:, 1], label='2 $\leftrightarrow$ 4')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_diag_eqv', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
# reproduce figure 5b
plt.figure(figsize=(2, 2))
plt.scatter(predictions_eqv_aug_13[:, 0], predictions_eqv_aug_13[:, 1], label='1 $\leftrightarrow$ 3')
plt.scatter(predictions_eqv_aug_24[:, 0], predictions_eqv_aug_24[:, 1], label='2 $\leftrightarrow$ 4')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_diag_eqv_aug', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
# reproduce figure 5c
plt.figure(figsize=(2, 2))
plt.scatter(predictions_fcnn_13[:, 0], predictions_fcnn_13[:, 1], label='1 $\leftrightarrow$ 3')
plt.scatter(predictions_fcnn_24[:, 0], predictions_fcnn_24[:, 1], label='2 $\leftrightarrow$ 4')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_diag_fcnn', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
# reproduce figure 5d
plt.figure(figsize=(2, 2))
plt.scatter(predictions_fcnn_aug_13[:, 0], predictions_fcnn_aug_13[:, 1], label='1 $\leftrightarrow$ 3')
plt.scatter(predictions_fcnn_aug_24[:, 0], predictions_fcnn_aug_24[:, 1], label='2 $\leftrightarrow$ 4')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_diag_fcnn_aug', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
# prepare model inputs and compute predictions for figure 6
increasing_rates = np.arange(0.0, 0.51, 0.01)
decreasing_rates = np.arange(0.5, -0.01, -0.01)
const_rates = 0.25 * np.ones(len(increasing_rates))
data_12 = np.c_[increasing_rates, decreasing_rates, const_rates, const_rates]
data_34 = np.c_[const_rates, const_rates, increasing_rates, decreasing_rates]
data_23 = np.c_[const_rates, increasing_rates, decreasing_rates, const_rates]
data_14 = np.c_[increasing_rates, const_rates, const_rates, decreasing_rates]
predictions_eqv_34 = model_eqv.predict(data_34)
predictions_eqv_12 = model_eqv.predict(data_12)
predictions_eqv_23 = model_eqv.predict(data_23)
predictions_eqv_14 = model_eqv.predict(data_14)
predictions_eqv_aug_34 = model_eqv_aug.predict(data_34)
predictions_eqv_aug_12 = model_eqv_aug.predict(data_12)
predictions_eqv_aug_23 = model_eqv_aug.predict(data_23)
predictions_eqv_aug_14 = model_eqv_aug.predict(data_14)
predictions_fcnn_34 = model_fcnn.predict(data_34)
predictions_fcnn_12 = model_fcnn.predict(data_12)
predictions_fcnn_23 = model_fcnn.predict(data_23)
predictions_fcnn_14 = model_fcnn.predict(data_14)
predictions_fcnn_aug_34 = model_fcnn_aug.predict(data_34)
predictions_fcnn_aug_12 = model_fcnn_aug.predict(data_12)
predictions_fcnn_aug_23 = model_fcnn_aug.predict(data_23)
predictions_fcnn_aug_14 = model_fcnn_aug.predict(data_14)

In [ ]:
# reproduce figure 6a
plt.figure(figsize=(2, 2))
plt.scatter(predictions_eqv_12[:, 0], predictions_eqv_12[:, 1], label='1 $\leftrightarrow$ 2')
plt.scatter(predictions_eqv_34[:, 0], predictions_eqv_34[:, 1], label='3 $\leftrightarrow$ 4')
plt.scatter(predictions_eqv_14[:, 0], predictions_eqv_14[:, 1], label='1 $\leftrightarrow$ 4', color='lime')
plt.scatter(predictions_eqv_23[:, 0], predictions_eqv_23[:, 1], label='2 $\leftrightarrow$ 3', color='silver')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_adj_eqv', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
# reproduce figure 6b
plt.figure(figsize=(2, 2))
plt.scatter(predictions_eqv_aug_12[:, 0], predictions_eqv_aug_12[:, 1], label='1 $\leftrightarrow$ 2')
plt.scatter(predictions_eqv_aug_34[:, 0], predictions_eqv_aug_34[:, 1], label='3 $\leftrightarrow$ 4')
plt.scatter(predictions_eqv_aug_14[:, 0], predictions_eqv_aug_14[:, 1], label='1 $\leftrightarrow$ 4', color='lime')
plt.scatter(predictions_eqv_aug_23[:, 0], predictions_eqv_aug_23[:, 1], label='2 $\leftrightarrow$ 3', color='silver')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_adj_eqv_aug', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
# reproduce figure 6c
plt.figure(figsize=(2, 2))
plt.scatter(predictions_fcnn_12[:, 0], predictions_fcnn_12[:, 1], label='1 $\leftrightarrow$ 2')
plt.scatter(predictions_fcnn_34[:, 0], predictions_fcnn_34[:, 1], label='3 $\leftrightarrow$ 4')
plt.scatter(predictions_fcnn_14[:, 0], predictions_fcnn_14[:, 1], label='1 $\leftrightarrow$ 4', color='lime')
plt.scatter(predictions_fcnn_23[:, 0], predictions_fcnn_23[:, 1], label='2 $\leftrightarrow$ 3', color='silver')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_adj_fcnn', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
# reproduce figure 6d
plt.figure(figsize=(2, 2))
plt.scatter(predictions_fcnn_aug_12[:, 0], predictions_fcnn_aug_12[:, 1], label='1 $\leftrightarrow$ 2')
plt.scatter(predictions_fcnn_aug_34[:, 0], predictions_fcnn_aug_34[:, 1], label='3 $\leftrightarrow$ 4')
plt.scatter(predictions_fcnn_aug_14[:, 0], predictions_fcnn_aug_14[:, 1], label='1 $\leftrightarrow$ 4', color='lime')
plt.scatter(predictions_fcnn_aug_23[:, 0], predictions_fcnn_aug_23[:, 1], label='2 $\leftrightarrow$ 3', color='silver')
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, -.9, .9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_adj_fcnn_aug', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
tmp = np.random.uniform(low=0.0, high=1.0, size=(1000000, 4))
idx_sum = np.sum(tmp, axis=1) <= 1
tmp = tmp[idx_sum, :]
idx_1 = (tmp[:, 0] > tmp[:, 1]) & (tmp[:, 0] > tmp[:, 2]) & (tmp[:, 0] > tmp[:, 3])
idx_2 = (tmp[:, 1] > tmp[:, 0]) & (tmp[:, 1] > tmp[:, 2]) & (tmp[:, 1] > tmp[:, 3])
idx_3 = (tmp[:, 2] > tmp[:, 0]) & (tmp[:, 2] > tmp[:, 1]) & (tmp[:, 2] > tmp[:, 3])
idx_4 = (tmp[:, 3] > tmp[:, 0]) & (tmp[:, 3] > tmp[:, 1]) & (tmp[:, 3] > tmp[:, 2])
data_1 = tmp[idx_1, :] / np.sum(tmp[idx_1, :], axis=1, keepdims=True)
data_2 = tmp[idx_2, :] / np.sum(tmp[idx_2, :], axis=1, keepdims=True)
data_3 = tmp[idx_3, :] / np.sum(tmp[idx_3, :], axis=1, keepdims=True)
data_4 = tmp[idx_4, :] / np.sum(tmp[idx_4, :], axis=1, keepdims=True)
predictions_eqv_1 = model_eqv.predict(data_1)
predictions_eqv_2 = model_eqv.predict(data_2)
predictions_eqv_3 = model_eqv.predict(data_3)
predictions_eqv_4 = model_eqv.predict(data_4)
predictions_eqv_aug_1 = model_eqv_aug.predict(data_1)
predictions_eqv_aug_2 = model_eqv_aug.predict(data_2)
predictions_eqv_aug_3 = model_eqv_aug.predict(data_3)
predictions_eqv_aug_4 = model_eqv_aug.predict(data_4)
predictions_fcnn_1 = model_fcnn.predict(data_1)
predictions_fcnn_2 = model_fcnn.predict(data_2)
predictions_fcnn_3 = model_fcnn.predict(data_3)
predictions_fcnn_4 = model_fcnn.predict(data_4)
predictions_fcnn_aug_1 = model_fcnn_aug.predict(data_1)
predictions_fcnn_aug_2 = model_fcnn_aug.predict(data_2)
predictions_fcnn_aug_3 = model_fcnn_aug.predict(data_3)
predictions_fcnn_aug_4 = model_fcnn_aug.predict(data_4)

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(predictions_eqv_1[:, 0], predictions_eqv_1[:, 1], alpha=0.03)
plt.scatter(predictions_eqv_2[:, 0], predictions_eqv_2[:, 1], alpha=0.03)
plt.scatter(predictions_eqv_3[:, 0], predictions_eqv_3[:, 1], color='lime', alpha=0.03)
plt.scatter(predictions_eqv_4[:, 0], predictions_eqv_4[:, 1], color='silver', alpha=0.03)
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, .9, -.9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_max_eqv', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(predictions_eqv_aug_1[:, 0], predictions_eqv_aug_1[:, 1], alpha=0.03)
plt.scatter(predictions_eqv_aug_2[:, 0], predictions_eqv_aug_2[:, 1], alpha=0.03)
plt.scatter(predictions_eqv_aug_3[:, 0], predictions_eqv_aug_3[:, 1], color='lime', alpha=0.03)
plt.scatter(predictions_eqv_aug_4[:, 0], predictions_eqv_aug_4[:, 1], color='silver', alpha=0.03)
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, .9, -.9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_max_eqv_aug', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(predictions_fcnn_1[:, 0], predictions_fcnn_1[:, 1], alpha=0.03)
plt.scatter(predictions_fcnn_2[:, 0], predictions_fcnn_2[:, 1], alpha=0.03)
plt.scatter(predictions_fcnn_3[:, 0], predictions_fcnn_3[:, 1], color='lime', alpha=0.03)
plt.scatter(predictions_fcnn_4[:, 0], predictions_fcnn_4[:, 1], color='silver', alpha=0.03)
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, .9, -.9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_max_fcnn', pad_inches=0.01, bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(2, 2))
plt.scatter(predictions_fcnn_aug_1[:, 0], predictions_fcnn_aug_1[:, 1], alpha=0.03)
plt.scatter(predictions_fcnn_aug_2[:, 0], predictions_fcnn_aug_2[:, 1], alpha=0.03)
plt.scatter(predictions_fcnn_aug_3[:, 0], predictions_fcnn_aug_3[:, 1], color='lime', alpha=0.03)
plt.scatter(predictions_fcnn_aug_4[:, 0], predictions_fcnn_aug_4[:, 1], color='silver', alpha=0.03)
plt.scatter([-.9, -.9, .9, .9], [-.9, .9, .9, -.9], s=50, facecolor='black', linewidth=0.0)
plt.text(-.75, .8, 'MFC1', fontsize=8)
plt.text(.42, .8, 'MFC2', fontsize=8)
plt.text(.42, -.89, 'MFC3', fontsize=8)
plt.text(-.75, -.89, 'MFC4', fontsize=8)
plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.grid(True)
plt.xticks([0])
plt.yticks([0])
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_aspect('equal', 'box')
plt.savefig('eusipco_predictions_max_fcnn_aug', pad_inches=0.01, bbox_inches='tight', dpi=200)